<a href="https://colab.research.google.com/github/serdarozsoy/comp541_project/blob/main/SimCLRjl_pretraining_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Julia on Colaboratory ##

[Colaboratory](https://colab.research.google.com) does not provide native support for the [Julia programming language](https://julialang.org). However, since Colaboratory gives you root access to the machine that runs your notebook (the *“runtime”* in Colaboratory terminology), we can install Julia support by uploading a specially crafted Julia notebook  – *this* notebook. We then install Julia and [IJulia](https://github.com/JuliaLang/IJulia.jl) ([Jupyter](https://jupyter.org)/Colaboratory notebook support) and reload the notebook so that Colaboratory detects and initiates what we installed.

In brief:

1. **Run the cell below**
2. **Reload the page**
3. **Edit the notebook name and start hacking Julia code below**

**If your runtime resets**, either manually or if left idle for some time, **repeat steps 1 and 2**.

### Acknowledgements ###

This hack by Pontus Stenetorp is an adaptation of [James Bradbury’s original Colaboratory Julia hack](https://discourse.julialang.org/t/julia-on-google-colab-free-gpu-accelerated-shareable-notebooks/15319/27), that broke some time in September 2019 as Colaboratory increased their level of notebook runtime isolation. There also appears to be CUDA compilation support installed by default for each notebook runtime type in October 2019, which shaves off a good 15 minutes or so from the original hack’s installation time. Adapted for Knet by Deniz Yuret.

In [ ]:
# @title Julia/IJulia installation cell
# @markdown run once (takes 30 seconds) and reload page before running the next cell

%%shell
wget 'https://julialang-s3.julialang.org/bin/linux/x64/1.6/julia-1.6.3-linux-x86_64.tar.gz' -O /tmp/julia.tar.gz
tar -x -f /tmp/julia.tar.gz -C /usr/local --strip-components 1
rm /tmp/julia.tar.gz
julia -e 'using Pkg; Pkg.add("IJulia")'

--2022-01-21 18:31:59--  https://julialang-s3.julialang.org/bin/linux/x64/1.6/julia-1.6.3-linux-x86_64.tar.gz
Resolving julialang-s3.julialang.org (julialang-s3.julialang.org)... 151.101.2.49, 151.101.66.49, 151.101.130.49, ...
Connecting to julialang-s3.julialang.org (julialang-s3.julialang.org)|151.101.2.49|:443... connected.
HTTP request sent, awaiting response... 302 gce internal redirect trigger
Location: https://storage.googleapis.com/julialang2/bin/linux/x64/1.6/julia-1.6.3-linux-x86_64.tar.gz [following]
--2022-01-21 18:31:59--  https://storage.googleapis.com/julialang2/bin/linux/x64/1.6/julia-1.6.3-linux-x86_64.tar.gz
Resolving storage.googleapis.com (storage.googleapis.com)... 173.194.203.128, 74.125.20.128, 74.125.135.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|173.194.203.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 113399344 (108M) [application/x-tar]
Saving to: ‘/tmp/julia.tar.gz’

/tmp/julia.tar.gz   100%[======

❗ RELOAD NOTEBOOK before running below cell

Add required packages:

In [5]:
# Knet installation cell: run after reloading/renaming notebook
# ENV["JULIA_CUDA_USE_BINARYBUILDER"]="false"  # Use this for faster installation, otherwise CUDA libraries will be downloaded
using Pkg; Pkg.add("Knet")
using Pkg; Pkg.add(["MLDatasets","PyCall","LinearAlgebra","JLD2"])
import Pkg; Pkg.add("CUDA")
# to download resnet file
import Pkg; Pkg.add("Git")

   Resolving package versions...
  No Changes to `~/.julia/environments/v1.6/Project.toml`
  No Changes to `~/.julia/environments/v1.6/Manifest.toml`
   Resolving package versions...
    Updating `~/.julia/environments/v1.6/Project.toml`
  [033835bb] + JLD2 v0.4.17
  No Changes to `~/.julia/environments/v1.6/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.6/Project.toml`
  No Changes to `~/.julia/environments/v1.6/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.6/Project.toml`
  No Changes to `~/.julia/environments/v1.6/Manifest.toml`


Download ResNet.jl from SimCLR.jl:

In [3]:
using Git
run(git(["clone", "https://github.com/serdarozsoy/SimCLR.jl.git"]))

Cloning into 'SimCLR.jl'...
/root/.julia/artifacts/33c5e3a13ad6427f86436f577c0edce1e468ac80/libexec/git-core/git-remote-https: /usr/local/bin/../lib/julia/libcurl.so.4: no version information available (required by /root/.julia/artifacts/33c5e3a13ad6427f86436f577c0edce1e468ac80/libexec/git-core/git-remote-https)


Process(setenv(`/root/.julia/artifacts/33c5e3a13ad6427f86436f577c0edce1e468ac80/bin/git clone https://github.com/serdarozsoy/SimCLR.jl.git`,["TF_FORCE_GPU_ALLOW_GROWTH=true", "CUDA_VERSION=11.1.1", "PATH=/root/.julia/artifacts/2a0fba617ae96fd7bec0e5f0981a2cc395998f08/bin:/root/.julia/artifacts/3831f541c8f90f52324b6020312a0cec7a8224aa/bin:/root/.julia/artifacts/75b2da9ed2ec48588460be0f3a8bb489212012e9/bin:/root/.julia/artifacts/33c5e3a13ad6427f86436f577c0edce1e468ac80/bin:/usr/local/nvidia/bin:/usr/local/cuda/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/tools/node/bin:/tools/google-cloud-sdk/bin:/opt/bin", "ENV=/root/.bashrc", "CLOUDSDK_CONFIG=/content/.config", "PWD=/", "GLIBCPP_FORCE_NEW=1", "__EGL_VENDOR_LIBRARY_DIRS=/usr/lib64-nvidia:/usr/share/glvnd/egl_vendor.d/", "DATALAB_SETTINGS_OVERRIDES={\"kernelManagerProxyPort\":6000,\"kernelManagerProxyHost\":\"172.28.0.3\",\"jupyterArgs\":[\"--ip", "COLUMNS=80"  …  "COLAB_GPU=1", "GIT_TEMPLATE_DIR=/root/.julia/artifac

Pretraining raw program to play at once:

In [6]:
using CUDA
using Knet
using MLDatasets
using PyCall
include("SimCLR.jl/resnet.jl")
using LinearAlgebra
using JLD2



# Load CIFAR-10 dataset
function load_cifar_dataset()
    xtrn, ytrn = CIFAR10.traindata(UInt8);
    # WxHxCxB -> HxWxCxB 
    xtrn = permutedims(xtrn, (2, 1, 3, 4))
    xtrn, ytrn
end


# Pycall implementation to use torchvision library for data augmentation 
py"""
from PIL import Image, ImageFilter, ImageOps
from torchvision import transforms

def init():
    return transforms.Compose(
                    [   transforms.ToPILImage(),
                        transforms.RandomResizedCrop(
                            (32, 32),
                            scale=(0.08, 1.0) 
                        ),
                        transforms.RandomApply(
                            [transforms.ColorJitter(brightness=0.4, contrast=0.4, saturation=0.2, hue=0.1)], p=0.8
                        ),
                        transforms.RandomGrayscale(p=0.2),
                        transforms.RandomHorizontalFlip(p=0.5),
                        transforms.ToTensor(),
                        transforms.Normalize((0.4914, 0.4822, 0.4465), (0.247, 0.243, 0.261)),
                    ]
                )


def transform_x(transform_model, x):
    return transform_model(x)
"""


function transform(x)
    return py"transform_x"(py"init"(), x)[:numpy]()
end


# load dataset
X_train, y_train = load_cifar_dataset();


# Model: ResNet + Projection Head
model_main = Sequential(ResNet(depth=18, in_ch=3, out_ch=512, cifar_stem=true), Linear(512,512,f=relu), Linear(512,128))


# Parameters
epochs = 10
learning_rate = 0.001

# Define optimizer
for p in params(model_main)
    p.opt = Adam(lr=learning_rate)
end

# Contrastive loss
function loss_contrastive(model, x, bs)

    LARGE_NUM = 1e9
    temperature = 0.5
    
    z = transpose(model(x))
    
    _atype = KnetArray{Float32}    
    norm_z = sqrt.(sum(abs2,z,dims=2))

    zx = z ./ norm_z

    z1 = zx[1:bs,:]
    z2 = zx[bs+1:bs*2,:]
    
    
    labels = convert(_atype, Array{Float64}(I, bs, bs*2))
    mask = convert(_atype, Array{Float64}(I, bs, bs)*LARGE_NUM)
    
    logits_aa = z1*transpose(z1)/temperature - mask 
    logits_bb = z2*transpose(z2)/temperature - mask 
    logits_ab = z1*transpose(z2)/temperature
    logits_ba = z2*transpose(z1)/temperature
    
    loss_a = sum(-labels.*logsoftmax([logits_ab logits_aa], dims=2))/bs
    loss_b = sum(-labels.*logsoftmax([logits_ba logits_bb], dims=2))/bs
    
    loss = loss_a + loss_b

    return loss
end


train_loss = zeros(epochs);
_atype = KnetArray{Float32}

# Training
for epoch in 1:epochs
    train_loss[epoch] = 0.0
    batch_count = 0
    dtrn = minibatch(X_train, y_train, 256, shuffle=true)
    for (batch_x, batch_y) in dtrn
        bs = size(batch_x)[4]
        batch_x1 = cat([transform(batch_x[:,:,:,i]) for i in 1:bs]..., dims=4)
        batch_x2 = cat([transform(batch_x[:,:,:,i]) for i in 1:bs]..., dims=4) 
        x2 = convert(_atype, permutedims(cat(batch_x1 , batch_x2, dims=4), (2, 3, 1, 4)))
        loss_main= @diff loss_contrastive(model_main, x2, bs)
        train_loss[epoch] += value(loss_main)
        batch_count += 1
        for p in params(model_main)
            g = grad(loss_main, p)
            update!(value(p), g, p.opt)
        end
    end
    train_loss[epoch] /= batch_count
    println("Train epoch: ", epoch ," ,loss: ", train_loss[epoch])
    flush(stdout)
    if mod(epoch, 100) == 0
      model_name = string("results/model_main_", epoch, ".jld2") 
      save(model_name, "model", model_main)
    end
end

This program has requested access to the data dependency CIFAR10.
which is not currently installed. It can be installed automatically, and you will not see this message again.

Dataset: The CIFAR-10 dataset
Authors: Alex Krizhevsky, Vinod Nair, Geoffrey Hinton
Website: https://www.cs.toronto.edu/~kriz/cifar.html
Reference: https://www.cs.toronto.edu/~kriz/learning-features-2009-TR.pdf

[Krizhevsky, 2009]
    Alex Krizhevsky.
    "Learning Multiple Layers of Features from Tiny Images",
    Tech Report, 2009.

The CIFAR-10 dataset is a labeled subsets of the 80
million tiny images dataset. It consists of 60000
32x32 colour images in 10 classes, with 6000 images
per class.

The compressed archive file that contains the
complete dataset is available for download at the
offical website linked above; specifically the binary
version for C programs. Note that using the data
responsibly and respecting copyright remains your
responsibility. The authors of CIFAR-10 aren't really
explicit about an

ERROR: Unexpected end of data : jl_WV3E7K-download


Train epoch: 1 ,loss: 10.947704990093525
Train epoch: 2 ,loss: 10.429761637174167
Train epoch: 3 ,loss: 10.219364239619328
Train epoch: 4 ,loss: 10.113775869516227
Train epoch: 5 ,loss: 10.029555403880584
Train epoch: 6 ,loss: 9.966238100100787
Train epoch: 7 ,loss: 9.907828815166766
Train epoch: 8 ,loss: 9.871267338288137
Train epoch: 9 ,loss: 9.834329849634415
Train epoch: 10 ,loss: 9.80773967351669
